In [1]:
import os
import json
import ofxparse
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv

jsons = []
path = Path(r"C:\Users\joao.manjabosco\Downloads\bluefit")
if path.exists():
    arquivos = path.iterdir()
    for arquivo in arquivos:
        if arquivo.suffix == '.ofx':
            for enc in ("utf-8", "latin-1", "cp1252"):
                try:
                    with open(arquivo, 'r', encoding=enc,) as f: 
                        ofx = ofxparse.OfxParser.parse(f)
                        break
                except Exception as e:
                    pass
            account = ofx.account
            statement = ofx.account.statement
            transactions = statement.transactions
            banco_nome = "Desconhecido"
            if account.institution and account.institution.organization:
                banco_nome = account.institution.organization
            for transaction in transactions:
                registro = {
                    "cod_banco": account.routing_number,
                    "banco": banco_nome,
                    "agencia": ofx.account.branch_id,
                    "num_conta": account.account_id,
                    "tipo_conta": ofx.account.account_type,
                    "data_inicio": statement.start_date.strftime('%d/%m/%Y'),
                    "data_fim": statement.end_date.strftime('%d/%m/%Y'),
                    "saldo": float(statement.balance),
                    "favorecido": transaction.payee, # favorecido ou “quem recebeu/pagou”
                    "tipo_transacao": transaction.type, # tipo da transação (por exemplo, “debit”, “credit”, etc.)
                    "data": transaction.date.strftime('%d/%m/%Y'), # data da transação
                    "valor": float(transaction.amount), # valor da transação
                    "descricao": transaction.memo,
                }
                jsons.append(registro)
            # Criar o arquivo JSON
            with open('./src/reports/raw.json', 'w', encoding='utf-8') as arquivo:
                json.dump(jsons, arquivo, ensure_ascii=False, indent=4)
                    

In [2]:
df = pd.DataFrame(jsons)
df["origem"] = df['tipo_transacao'].map({'credit': 'CAR', 'debit': 'CAP', 'dep': 'CAR'})

display(df)

,cod_banco,banco,agencia,num_conta,tipo_conta,data_inicio,data_fim,saldo,favorecido,tipo_transacao,data,valor,descricao,origem
0,001,Banco do Brasil S/A,,113159-1,CHECKING,31/07/2025,31/08/2025,0.00,,dep,01/08/2025,257.76,REDE VENDAS VISA CRÉDITO,CAR
1,001,Banco do Brasil S/A,,113159-1,CHECKING,31/07/2025,31/08/2025,0.00,,dep,01/08/2025,218.00,REDE VENDAS MASTER CRÉDIT,CAR
2,001,Banco do Brasil S/A,,113159-1,CHECKING,31/07/2025,31/08/2025,0.00,,dep,01/08/2025,12918.18,REDE VENDAS MASTER CRÉDIT,CAR
3,001,Banco do Brasil S/A,,113159-1,CHECKING,31/07/2025,31/08/2025,0.00,,dep,01/08/2025,609.42,REDE CRÉDITO,CAR
4,001,Banco do Brasil S/A,,113159-1,CHECKING,31/07/2025,31/08/2025,0.00,,dep,01/08/2025,6697.01,REDE VENDAS VISA CRÉDITO,CAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1424,0341,Desconhecido,,4308091703,CHECKING,01/09/2025,30/09/2025,224132.32,,debit,30/09/2025,-100.00,PIX ENVIADO MILTON ALVES DE ARAUJO 324.171.951-00,CAP
1425,0341,Desconhecido,,4308091703,CHECKING,01/09/2025,30/09/2025,224132.32,,debit,30/09/2025,-858.03,PIX ENVIADO ANAIR BORGES DE SOUSA 194.195.621-15,CAP
1426,0341,Desconhecido,,4308091703,CHECKING,01/09/2025,30/09/2025,224132.32,,debit,30/09/2025,-165.00,TAR CONTR/RENOV CTA GAR,CAP
1427,0341,Desconhecido,,4308091703,CHECKING,01/09/2025,30/09/2025,224132.32,,debit,30/09/2025,-300.00,SISPAG TRANSF CC ITAU,CAP


In [3]:
import re
import os
import ast
import json
import math
import asyncio
import pandas as pd
from typing import Optional, Literal
from pydantic import BaseModel, Field
from tqdm import tqdm  # usando tqdm normal ao invés de tqdm.notebook
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Configurar nest_asyncio logo no início
import nest_asyncio
nest_asyncio.apply()

load_dotenv()

# ==== Configurações ====
LLM_MODEL = "gpt-4o-mini"
LLM_TEMPERATURE = 0
MAX_CONCURRENCY = 6

# ==== Classes permitidas (do arquivo regra.json) ====
with open('./src/prompts/regra.json', 'r', encoding='utf-8') as f:
    regra = json.load(f)
    CLASSES_PERMITIDAS = regra["contexto"]["classes_permitidas"]

# ==== Modelo de dados para classificação ====
class ClassificacaoTransacao(BaseModel):
    classificacao_sugerida: Literal[tuple(CLASSES_PERMITIDAS)] = Field(..., description="Classificação da transação.")
    explicacao: str = Field(..., description="Explicação da classificação.")

# ==== Template do prompt ====
SYSTEM_PROMPT = f"""
Você é um analista financeiro especializado em classificar transações bancárias.

REGRAS DE CLASSIFICAÇÃO:
{json.dumps(regra["contexto"]["instrucoes_gerais"], indent=2, ensure_ascii=False)}

CLASSES PERMITIDAS:
{json.dumps(CLASSES_PERMITIDAS, indent=2, ensure_ascii=False)}
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("human", """Classifique a transação a seguir:
    
Descrição: {descricao}
Origem: {origem} (CAR=Crédito, CAP=Débito)
Valor: R$ {valor}
""")
])

# ==== Criação do modelo ====
llm = ChatOpenAI(model=LLM_MODEL, temperature=LLM_TEMPERATURE)
chain = (prompt | llm.with_structured_output(ClassificacaoTransacao))

In [ ]:
# ==== Funções auxiliares ====
def regras_pre_classificacao(row: pd.Series) -> Optional[dict]:
    """Aplica regras básicas antes de chamar a IA"""
    desc = str(row.get("descricao", "") or "").lower()
    
    # Regras de pré-classificação baseadas no arquivo regra.json    
    if "pix" in desc and (" BODY STATION ACADEMIA " in desc or " J E MADEIRA A " in desc):
        if row["origem"] == "CAR":
            return {"classificacao_sugerida": "(+) Transferencia entre Contas", "explicacao": "Transferência entre contas identificada"}
        else:
            return {"classificacao_sugerida": "(-) Transferencia entre Contas", "explicacao": "Transferência entre contas identificada"}
        
    if "pix" in desc or "ted" in desc:
        if row["origem"] == "CAP":
            return {"classificacao_sugerida": "(-) Transferencia", "explicacao": "Transferência identificada (saída)"}
        else:
            return {"classificacao_sugerida": "(+) Transferencia", "explicacao": "Transferência identificada (entrada)"}
            
    if "rede" in desc or "cartao" in desc or "cartão" in desc:
        return {"classificacao_sugerida": "Receita", "explicacao": "Recebimento via cartão/maquininha"}
        
    if "seguro" in desc:
        return {"classificacao_sugerida": "Seguro", "explicacao": "Seguro identificado na descrição"}
        
    if "consorcio" in desc or "consórcio" in desc:
        return {"classificacao_sugerida": "Consorcio", "explicacao": "Consórcio identificado na descrição"}
        
    if any(k in desc for k in ["ourocap", "rende facil", "rende fácil", "invest"]):
        return {"classificacao_sugerida": "Investimento", "explicacao": "Investimento identificado na descrição"}
    
    if "RENDE FÁCIL" in desc:
        if row["origem"] == "CAR":
            return {"classificacao_sugerida": "Resgate de Aplicação", "explicacao": "Resgate de aplicação financeira identificado"}
        else:
            return {"classificacao_sugerida": "Aplicação financeira", "explicacao": "Aplicação financeira identificada"}
    
    return None

async def classificar_transacao(row: pd.Series) -> dict:
    """Classifica uma única transação"""
    # Tenta regras pré-definidas primeiro
    pre_class = regras_pre_classificacao(row)
    if pre_class:
        return pre_class
        
    # Se não conseguiu classificar com regras, usa a IA
    try:
        resultado = await chain.ainvoke({
            "descricao": row["descricao"],
            "origem": row["origem"],
            "valor": row["valor"]
        })
        return resultado.model_dump()
    except Exception as e:
        print(f"❌ Erro ao classificar transação: {e}")
        return {
            "classificacao_sugerida": "Nao classificado",
            "explicacao": f"Erro na classificação: {str(e)}"
        }

async def _gather_limit(coros, limit: int):
    """Executa coroutines com limite de concorrência"""
    sem = asyncio.Semaphore(limit)
    async def run(c):
        async with sem:
            return await c
    return await asyncio.gather(*(run(c) for c in coros))

async def classificar_transacoes_df(df: pd.DataFrame) -> pd.DataFrame:
    """Classifica todas as transações do DataFrame"""
    df = df.copy()
    
    # Inicializa colunas se não existirem
    if "classificacao_sugerida" not in df.columns:
        df["classificacao_sugerida"] = None
    if "explicacao" not in df.columns:
        df["explicacao"] = None
    
    # Identifica transações não classificadas
    mask_sem_class = df["classificacao_sugerida"].isna()
    indices = df.index[mask_sem_class]
    
    if len(indices) > 0:
        # Cria tasks para cada transação
        tasks = [classificar_transacao(df.loc[i]) for i in indices]
        
        # Processa em chunks para não sobrecarregar
        resultados = []
        for start in tqdm(range(0, len(tasks), 50), desc="Classificando transações"):
            chunk = tasks[start:start + 50]
            out = await _gather_limit(chunk, MAX_CONCURRENCY)
            resultados.extend(out)
        
        # Atualiza o DataFrame com os resultados
        for idx, res in zip(indices, resultados):
            df.loc[idx, ["classificacao_sugerida", "explicacao"]] = [
                res["classificacao_sugerida"],
                res["explicacao"]
            ]
    
    return df

In [5]:
# ==== Executa a classificação ====
def classificar(df: pd.DataFrame) -> pd.DataFrame:
    """Função principal para classificar transações"""
    loop = asyncio.get_event_loop()
    return loop.run_until_complete(classificar_transacoes_df(df))

print("🔄 Iniciando classificação...")
df_classificado = classificar(df)
print("✅ Classificação concluída!")
display(df_classificado)

🔄 Iniciando classificação...


Classificando transações: 100%|██████████| 29/29 [02:11<00:00,  4.53s/it]


✅ Classificação concluída!


,cod_banco,banco,agencia,num_conta,tipo_conta,data_inicio,data_fim,saldo,favorecido,tipo_transacao,data,valor,descricao,origem,classificacao_sugerida,explicacao
0,001,Banco do Brasil S/A,,113159-1,CHECKING,31/07/2025,31/08/2025,0.00,,dep,01/08/2025,257.76,REDE VENDAS VISA CRÉDITO,CAR,Receita,Recebimento via cartão/maquininha
1,001,Banco do Brasil S/A,,113159-1,CHECKING,31/07/2025,31/08/2025,0.00,,dep,01/08/2025,218.00,REDE VENDAS MASTER CRÉDIT,CAR,Receita,Recebimento via cartão/maquininha
2,001,Banco do Brasil S/A,,113159-1,CHECKING,31/07/2025,31/08/2025,0.00,,dep,01/08/2025,12918.18,REDE VENDAS MASTER CRÉDIT,CAR,Receita,Recebimento via cartão/maquininha
3,001,Banco do Brasil S/A,,113159-1,CHECKING,31/07/2025,31/08/2025,0.00,,dep,01/08/2025,609.42,REDE CRÉDITO,CAR,Receita,Recebimento via cartão/maquininha
4,001,Banco do Brasil S/A,,113159-1,CHECKING,31/07/2025,31/08/2025,0.00,,dep,01/08/2025,6697.01,REDE VENDAS VISA CRÉDITO,CAR,Receita,Recebimento via cartão/maquininha
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1424,0341,Desconhecido,,4308091703,CHECKING,01/09/2025,30/09/2025,224132.32,,debit,30/09/2025,-100.00,PIX ENVIADO MILTON ALVES DE ARAUJO 324.171.951-00,CAP,(-) Transferencia,Transferência identificada (saída)
1425,0341,Desconhecido,,4308091703,CHECKING,01/09/2025,30/09/2025,224132.32,,debit,30/09/2025,-858.03,PIX ENVIADO ANAIR BORGES DE SOUSA 194.195.621-15,CAP,(-) Transferencia,Transferência identificada (saída)
1426,0341,Desconhecido,,4308091703,CHECKING,01/09/2025,30/09/2025,224132.32,,debit,30/09/2025,-165.00,TAR CONTR/RENOV CTA GAR,CAP,Seguro,"A descrição contém a palavra 'RENOV', que suge..."
1427,0341,Desconhecido,,4308091703,CHECKING,01/09/2025,30/09/2025,224132.32,,debit,30/09/2025,-300.00,SISPAG TRANSF CC ITAU,CAP,(-) Transferencia,A descrição indica uma transferência que sai d...


In [6]:
df_classificado.to_excel('./src/reports/classified_transactions.xlsx', index=False)